# Предсказание структуры

## 0. Установка пакетов

### 0.1 Локально

Команды для установки, набрать в консоли (при необходимости предварительно активировав нужный environment)

```
pip install biopython
conda install -c salilab modeller
```
После этого моделлер предложит отредактировать файл config.py (полный путь он выдаст) и добавить туда пароль. Добавляем слово MODELIRANJE .

### 0.2 В колабе (надеюсь, не понадобится)

In [ ]:
! pip install biopython
! cd /tmp/ ; wget https://salilab.org/modeller/10.1/modeller_10.1-1_amd64.deb
! env KEY_MODELLER="MODELIRANJE" dpkg -i /tmp/modeller_10.1-1_amd64.deb
! echo -e "install_dir = r'/usr/lib/modeller10.1'\nlicense = 'MODELIRANJE'" > /usr/lib/modeller10.1/modlib/modeller/config.py

import sys
sys.path.append('/usr/lib/python3.9/dist-packages')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.7 MB/s eta 0:00:00


Если пимол запускается локально, но не через ноутбук, или проблемы с установкой моделлера, то моделлер можно запускать в колабе, результаты расчетов скачивать и открывать пимолом.

## 1. Постановка задачи

Имеется структура комплекса антитела с RBD доменом коронавируса (https://www.rcsb.org/structure/7SIX)

Попробуем построить модель комплекса с помощью 2 методов: modeller и alphafold2. Проверим, насколько хорошо построенные модели соответствуют настоящей структуре.

In [ ]:
from Bio import PDB, ExPASy, SwissProt, SeqIO
from IPython.display import Image
import os


Загрузим последовательность RBD домена из Swissprot с помощью биопитона

In [ ]:
record = SwissProt.read(ExPASy.get_sprot_raw('SPIKE_SARS2')) # скачиваем запись о спайк-белке
rbd_seq=record.sequence[318:541] #  вырезаем из сиквенса белка RBD
rbd_seq

Последовательности антитела (тяжелой и легкой цепей) возьмем из файла со структурой

In [ ]:
pdbl = PDB.PDBList()
target=pdbl.retrieve_pdb_file("7SIX",file_format='pdb',pdir='.') # скачиваем файл структуры

In [ ]:
for record in SeqIO.parse("pdb7six.ent", "pdb-seqres"): # парсим его шапку и берем оттуда нужные сиквенсы
    if record.name=='7SIX:H':
        heavy_seq=str(record.seq)
    elif record.name=='7SIX:L':
        light_seq=str(record.seq)

In [ ]:
heavy_seq

In [ ]:
light_seq

Посмотрим, что мы собираемся предсказывать

In [ ]:
# для тех, у кого все работает
import __main__
__main__.pymol_argv = [ 'pymol', '-x' ]

import pymol
pymol.finish_launching()

In [ ]:
from pymol import cmd
cmd.do('bg white')

In [ ]:
cmd.load(target)

#cmd.png('img_1.png')
#Image('img_1.png')

# заметим, что в структуре 2 цепи антитела (легкая и тяжелая) и 2 RBD домена, поскольку данное антитело связывается с RBD в двух местах.

## 2. AlphaFold2

Предскажем структуру комплекса с помощью alphafold2 https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb. Для этого идем по ссылке, заполняем поле query_sequence целевой последовательностью (при необходимости используем разделитель цепей ":") и в меню сверху жмем Runtime > Run all. Расчеты идут минут 15, после чего архив с результатами автоматически скачивается.

Распакуем архив с результатами и посмотрим, что там

In [ ]:
! unzip < имя архива >

In [ ]:
os.listdir( < имя распакованного архива > )

In [ ]:
os.rename( < имя распакованного архива/ имя нужного файла > ,
           'alphafold.pdb') # из распакованного архива вытащим файл с моделью и переименуем

Откроем получившуюся структуру в пимоле

In [ ]:
cmd.load(...)

Наложим на настоящую структуру и проверим, насколько верно альфафолд предсказал положение антитела

In [ ]:
cmd.alignto(...) # попробуйте наложить поочередно на оба положения RBD домена
cmd.orient(...)

Приблизим участок взаимодействия антитела и RBD (вручную), покажем аминокислоты в виде sticks, оценим, насколько реалистично предсказано взаимодействие.

In [ ]:
...

Выводы:

...

...

## 3. Modeller

Для той же задачи используем метод гомологичного моделирования

In [ ]:
import modeller
from modeller import automodel


In [ ]:
env=modeller.environ()
#env.io.hetatm=True # это если нужно сохранить гетероатомы

Создадим объект выравнивание и добавим туда наши последовательности (с разделителем цепей "/")

In [ ]:
aln=modeller.alignment(env)

aln.append_sequence('/'.join([rbd_seq, heavy_seq, light_seq]))
aln[0].code='rbd'
aln

Для работы моделлеру нужна структура, по образцу который будет строится новая модель. Возьмем любую структуру комплекса RBD с антителом, например https://www.rcsb.org/structure/7DET

In [ ]:
pdbl = PDB.PDBList()
template=pdbl.retrieve_pdb_file("7DET",file_format='pdb',pdir='.')
template

Загрузим эту структуру в наше выравнивание

In [ ]:
mdl = modeller.model(env, file=template, model_segment=('FIRST:A', 'LAST:B'))

# заметим, что в этой структуре только 2 цепи: RBD и антитела.
# антитело здесь это сшитые между собой тяжелая и легкая цепи.
# так что наши 3 цепи будут выравниваться с 2 цепями образца.

aln.append_model(mdl, atom_files=template, align_codes='7det')

Рассчитаем выравнивание и сохраним в файл

In [ ]:
aln.salign()
aln.write(file='lys.ali', alignment_format='PIR')

Рассчитаем модель

In [ ]:
a = automodel.automodel(env, alnfile='lys.ali', knowns= '7det', sequence = 'rbd' )

a.starting_model = 1
a.ending_model = 2
a.make()

In [ ]:
os.listdir('.')

Загрузим полученную модель в пимол и посмотрим, насколько хорошо предсказалась структура

In [ ]:
...

In [ ]:
...

In [ ]:
...

Выводы:

...

...

## 4. Домашнее задание

0. Самостоятельно сделать прак (кто не пришел).
1. Построить структуру комплекса с помощью любого другого метода (ноутбуки для запуска доступны по ссылке https://github.com/sokrypton/ColabFold). В пимоле рассмотреть участок взаимодействия антитела и RBD, сравнить с оригинальной структурой, сделать выводы.
2. Построить структуру лизоцима (https://www.uniprot.org/uniprotkb/P26836/entry) с помощью alphafold2 и создать модель его комплекса с олигосахаридом с помощью modeller (в качестве template использовать комплекс https://www.rcsb.org/structure/1LMP). Подсказки: https://vsb.fbb.msu.ru/share/modeller/node18.html

